## First attempt

In [ ]:
import numpy as np
import os 

# Function to read obj file and compute the number of vertices

def read_obj_file(file_path):
    vertices = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('v '):
                components = line.split()[1:]
                vertex = [float(comp) for comp in components]
                vertices.append(vertex)

    return np.array(vertices)

# Sample
sample_cad_model = read_obj_file("/home/hrishi/Programming/Python/CAD-DR/abc-dataset/00030037_0ef34aa1b15748a5b4ad7c0e_trimesh_027.obj")
print(sample_cad_model)
print(sample_cad_model.shape)

In [ ]:
import os

directory_path = '/home/hrishi/Programming/Python/CAD-DR/abc-dataset/'
file_paths = [os.path.join(directory_path, filename) for filename in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, filename))]
print(len(file_paths))
print(file_paths[0])

In [ ]:
import pandas as pd

dataset = []
for i in file_paths:
    data = read_obj_file(str(i))
    data = pd.DataFrame(data)
    dataset.append(data)

print(dataset)
print(len(dataset))

In [ ]:
# Finding the number of vertices of each model in the dataset
max_vertices = 0
min_vertices = 1000000
for filename in os.listdir("abc-dataset"):
    vertices = len(read_obj_file("abc-dataset/" + str(filename)))
    # print(vertices) - can be used if we wish to see number of vertices of each model
    if (vertices > max_vertices):
        max_vertices = vertices
    if (vertices < min_vertices):
        min_vertices = vertices

In [ ]:
# Maximum number of vertices
print(max_vertices)
print(min_vertices)
print(dataset[0])

In [ ]:
import tensorflow as tf

if tf.test.is_gpu_available():
    print("GPU is available and being used by TensorFlow.")

In [ ]:
import numpy as np
import pandas as pd
from keras.layers import Input, Dense
from keras.models import Model

dataframes = dataset
data = np.vstack([df.values for df in dataframes])

desired_dimensions = 1900

num_encoding_layers = 10
num_decoding_layers = 10

input_layer = Input(shape=(data.shape[1],))
encoded = input_layer

for _ in range(num_encoding_layers):
    encoded = Dense(512, activation='relu')(encoded)

encoded = Dense(desired_dimensions, activation='relu')(encoded)
decoded = encoded
for _ in range(num_decoding_layers):
    decoded = Dense(512, activation='relu')(decoded)

decoded = Dense(data.shape[1], activation='linear')(decoded)
autoencoder = Model(input_layer, decoded)
encoder = Model(input_layer, encoded)
autoencoder.compile(optimizer='adam', loss='mse')


reduced_dataframes = []

for epoch in range(4):
    autoencoder.fit(data, data, batch_size=60)

    epoch_reduced_dataframes = []

    for i, dataframe in enumerate(dataframes):
        sample_data = dataframe.values
        latent_representation = encoder.predict(sample_data)
        reconstructed_data = autoencoder.predict(sample_data)

        epoch_reduced_dataframes.append(pd.DataFrame(latent_representation))

    reduced_dataframes.append(epoch_reduced_dataframes)


epoch_to_print = 0
for i, reduced_dataframe in enumerate(reduced_dataframes[epoch_to_print]):
    print(
        f'Reduced Dimensionality DataFrame (Epoch {epoch_to_print+1}, DataFrame {i+1}):')
    print(reduced_dataframe)

autoencoder.save_weights('autoencoder_weights.h5')

In [ ]:
# training the autoencoder

import numpy as np
import pandas as pd
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.regularizers import l2

dataframes = dataset
data = np.vstack([df.values for df in dataframes])

desired_dimensions = 1900

num_encoding_layers = 10
num_decoding_layers = 10

input_layer = Input(shape=(data.shape[1],))
encoded = input_layer

for _ in range(num_encoding_layers):
    encoded = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(encoded)

encoded = Dense(desired_dimensions, activation='relu')(encoded)
decoded = encoded

for _ in range(num_decoding_layers):
    decoded = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(decoded)

decoded = Dense(data.shape[1], activation='sigmoid')(decoded)
autoencoder = Model(input_layer, decoded)
encoder = Model(input_layer, encoded)

optimizer = Adam(learning_rate=0.001)
autoencoder.compile(optimizer=optimizer, loss='mse', batch_size=15)

In [ ]:
# using the ae on other data

reduced_dimensionality_representations = []
for dataframe in dataframes:
  sample_data = dataframe.values
  latent_representation = encoder.predict(sample_data)
  reduced_dimensionality_representations.append(latent_representation)

## Second attempt